<a href="https://colab.research.google.com/github/awal015/CapstoneStep4/blob/main/Board_Game_Recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
threnjen_board_games_database_from_boardgamegeek_path = kagglehub.dataset_download('threnjen/board-games-database-from-boardgamegeek')

print('Data source import complete.')


100%|██████████| 148M/148M [00:07<00:00, 20.0MB/s]

Extracting files...


Data source import complete.


** $Board\ Game\ Database\ from\ Board\ GameGeek$**

# $General\ Overview$

### Dataset

* Data was taken from Kaggle - https://www.kaggle.com/threnjen/board-games-database-from-boardgamegeek?select=user_ratings.csv
* This dataset Feature-rich multi-table dataset full of interesting information about Board Games which can be used for tasks such as exploratory EDA, predictive modeling, or recommender systems.


### Project Modules & Packages

In [2]:
!pip install FuzzyWuzzy
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357286 sha256=a4bfc59bc8b25cc6db71bf8097f7b6dcb91231946f5d941c336801a8d0d267e9
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [3]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score ,f1_score ,roc_auc_score ,log_loss
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV ,StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

warnings.filterwarnings('ignore')
%matplotlib inline

from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process

from scipy.spatial import distance

from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split as sup_tt_split

Kaggke API
**remember to upload kaggke json credatinatials

In [4]:
!pip install -q kaggle

In [5]:
#upload file and then -->
#create kaggle folder
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [6]:
#copy json file to kaggle folder
! cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [7]:
#dowload board game zip file
!kaggle datasets download -d threnjen/board-games-database-from-boardgamegeek

Dataset URL: https://www.kaggle.com/datasets/threnjen/board-games-database-from-boardgamegeek
License(s): CC-BY-SA-3.0
 98% 145M/148M [00:05<00:00, 28.9MB/s]
100% 148M/148M [00:05<00:00, 26.4MB/s]


In [12]:
#unzip dataset files
!unzip /content/board-games-database-from-boardgamegeek.zip

Archive:  /content/board-games-database-from-boardgamegeek.zip
replace artists_reduced.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: artists_reduced.csv     
  inflating: bgg_data_documentation.txt  
  inflating: designers_reduced.csv   
  inflating: games.csv               
  inflating: mechanics.csv           
  inflating: publishers_reduced.csv  
  inflating: ratings_distribution.csv  
  inflating: subcategories.csv       
  inflating: themes.csv              
  inflating: user_ratings.csv        


In [13]:
#set max columns to
pd.set_option('display.max_columns', 20 )
pd.set_option('display.max_rows' , 40)
pd.options.display.max_rows

40

### Load CSV file to Pandas DataFrame

In [16]:
#read csv's
games = pd.read_csv('../input/board-games-database-from-boardgamegeek/games.csv')
mechanics = pd.read_csv('../input/board-games-database-from-boardgamegeek/mechanics.csv')
themes = pd.read_csv('../input/board-games-database-from-boardgamegeek/themes.csv')
subcategories = pd.read_csv('../input/board-games-database-from-boardgamegeek/subcategories.csv')
rank_dist = pd.read_csv('../input/board-games-database-from-boardgamegeek/ratings_distribution.csv')
user_rank = pd.read_csv('../input/board-games-database-from-boardgamegeek/user_ratings.csv')
#games.head()

FileNotFoundError: [Errno 2] No such file or directory: '../content/board-games-database-from-boardgamegeek/games.csv'

In [ ]:
games.head(3)

Merge df

In [ ]:
features = games = pd.read_csv('../input/board-games-database-from-boardgamegeek/games.csv')
features = pd.concat( [features.BGGId , features.iloc[:,-8:-1] ] ,axis =1 )
features = features.merge(subcategories, how="left" ,left_on = 'BGGId' ,right_on= 'BGGId',  suffixes=("_left", "_right"))
features = features.merge(themes, how="left" ,left_on = 'BGGId' ,right_on= 'BGGId',  suffixes=("_left", "_right"))
features = features.merge( mechanics , how="left" ,left_on = 'BGGId' ,right_on= 'BGGId',  suffixes=("_left", "_right"))
features.shape

# Most Popular Games

In [ ]:
# bringing most pop games for new users
games_pop = games.copy()
games_pop['game_popularty'] = games_pop['AvgRating'] /10 * games_pop['NumOwned']
top_ten_pop_games = games_pop.sort_values(by='game_popularty',ascending = False).head(10)
top_ten_pop_games = top_ten_pop_games[['BGGId','Name','Description','ImagePath']]
top_ten_pop_games = top_ten_pop_games.reset_index(drop = True)
top_ten_pop_games

# Collaborative filtering - Rate by User Ratings

![image.png](attachment:055830b3-f9b1-49a3-92d7-4c7d59101f88.png)

![image.png](attachment:29f042cf-ce66-407a-ab6b-5f09802ad5bb.png)

In [ ]:
#number of users in user ratings
user_rank.Username.nunique()

In [ ]:
#choose user to suggest to
active_user_id = 'zzzvone'
active_user_ratings = user_rank[user_rank['Username'] == active_user_id]

print(active_user_ratings.shape)
active_user_ratings.head()

In [ ]:
#get active_user_games and users that played same games as the active user
active_user_games = active_user_ratings['BGGId']
common_ratings = user_rank[user_rank['BGGId'].isin(active_user_games)]

print(f"Number of users that have at least one game in common with the active \
user: {common_ratings.Username.nunique()} users")

print(common_ratings.shape)
common_ratings.sample(5)

In [ ]:
#filter users that have rated at least 'min_common_items'(15) from 'active user' games
min_common_items = 15
potential_neighbors = common_ratings.groupby('Username').filter(lambda grp: len(grp) >= min_common_items)

print(f"{potential_neighbors.Username.nunique()} users")

print(potential_neighbors.shape)
potential_neighbors.sample(5)

In [ ]:
#remove 'active_user_id' from potential_neighbors
print(potential_neighbors.shape)
potential_neighbors = potential_neighbors.loc[potential_neighbors.Username != active_user_id]
print(potential_neighbors.shape)

In [ ]:
### view neighbor for example and compare to 'active user ratings'
# neighbor_id = 'Thesp'

# neighbor_ratings = potential_neighbors.loc[potential_neighbors.Username == neighbor_id, :]

# print(neighbor_ratings.shape)
# neighbor_ratings

# negihbor_ratings_compare = neighbor_ratings.merge(active_user_ratings, on='BGGId', how='inner', suffixes=('_neighbor', '_active'))
# negihbor_ratings_compare

In [ ]:
#take neighbor_ratings df return similarity score
def calc_similarity(neighbor_ratings , active_user_ratings):
    negihbor_ratings_compare = neighbor_ratings.merge(active_user_ratings,
         on='BGGId', how='inner', suffixes=('_neighbor', '_active'))

    cos_distance = distance.cosine(negihbor_ratings_compare['Rating_active'],
                                   negihbor_ratings_compare['Rating_neighbor'])
    return 1 - cos_distance

In [ ]:
#Apply calc_similarity function on user in 'potential_neighbors'
similarities = potential_neighbors.groupby('Username').apply(calc_similarity ,(active_user_ratings) )
similarities.name = 'Similarity'
similarities.shape

In [ ]:
# get active user neighbors where min_similarity_score > X
min_similarity_score = 0.96
relevant_neighbors = similarities[similarities > min_similarity_score].index

print(len(relevant_neighbors))
relevant_neighbors

In [ ]:
#get games suggestion where relevant_neighbors rate the game
    #and the active user did NOT rate the game
recommend_ratings = user_rank.loc[user_rank['Username'].isin(relevant_neighbors) &   # only games that were rated by the active user neighbors
                                   ~user_rank['BGGId'].isin(active_user_games)]  # only games that the active user did not see

print(recommend_ratings.shape)
recommend_ratings.head()

In [ ]:
# filter 'recommend_ratings' where number of ratings by
# the relevant neighbors on a specifc game > 'min_neighbors_ratings' (7)
# (without the active user)

#why ? - in order to later get a valid Weighted average score
min_neighbors_ratings = 7

print(recommend_ratings.shape)
recommend_ratings = recommend_ratings.groupby('BGGId').filter(
    lambda grp: len(grp) > min_neighbors_ratings)
print(recommend_ratings.shape)
recommend_ratings.BGGId.nunique()
recommend_ratings.info()

In [ ]:
def calc_game_score(game_ratings):
    game_ratings = game_ratings.join(similarities, on='Username')
    return (game_ratings['Rating'] *
            game_ratings['Similarity']).sum() / \
            game_ratings['Similarity'].sum()

In [ ]:
game_scores = recommend_ratings.groupby('BGGId').apply(calc_game_score)
game_scores.name = 'Score'
game_scores.head()

In [ ]:
#option 2 ################ Calc game rating weighted avg (with similarity)
def calc_game_score(game_ratings , similarities):
    game_ratings = game_ratings.join(similarities, on='Username')
    return (game_ratings['Rating'] *
            game_ratings['Similarity']).sum() / \
            game_ratings['Similarity'].sum()

In [ ]:
# apply calc_game_score on recommend_ratings
game_scores = recommend_ratings.groupby('BGGId').apply(calc_game_score ,(similarities) )
game_scores.name = 'Score'
game_scores.head()

In [ ]:
# ### example weighted avg rating score
# example = recommend_ratings[recommend_ratings.BGGId == 40819]
# example = example.join(similarities, on='Username')
#print(game_scores[40819])
# example

In [ ]:
#select top 5 games suggestion by game score (rating weighted avg)
n_recommendations = 5
final_recommendation = game_scores.sort_values(ascending=False)[:n_recommendations]
final_recommendation= final_recommendation.to_frame('score').reset_index()
final_recommendation

In [ ]:
#merge top 5 games suggestion with game details
top_picks= final_recommendation.merge( games[['BGGId','Name','Description']] , on='BGGId', how='left')
top_picks

# Game Recommendation -  Content Based

![image.png](attachment:78a53c29-3c35-4ed2-b589-3183acd0d82f.png)

Check Feature usage

In [ ]:
#are there features that are not in use ? -- no
test = features.iloc[: ,1:]
sum_row = test.sum(axis = 0)
print(sum_row[sum_row == 0])

#are there agmes where all features are 0 ? --> 136 games
test = features.iloc[: ,1:]
sum_row = test.sum(axis = 1)
sum_row[sum_row == 0].count()

In [ ]:
#game feature propotion - the precentage of marked features in every game
features.iloc[:,1:].mean(axis = 1).hist(bins = 30)

In [ ]:
#convert to sparse matrix element
features_sparse = csr_matrix(features.iloc[:,1:].values) #BGGId left out

Model Object

In [ ]:
#define knn model parameters
model_knn = NearestNeighbors(metric= 'cosine' , algorithm= 'brute' , n_neighbors= 11 )
model_knn

In [ ]:
#fit the model
model_knn.fit(features_sparse)

##Get Recommendation by giving game name

Get Game Name

In [ ]:
#try to suggest exact names
#enter_game_name = 'catan'
enter_game_name = input('Please enter board game name :')
games_name_suggest = process.extract(enter_game_name , games.Name , limit = 10)
print(f'Chose game from the list:')
for game in games_name_suggest:
    print(game[0])
enter_game_name = input('chosen game is :')

In [ ]:
exact_name , score , idx = process.extractOne( enter_game_name , games.Name )
print(f' found the game : "{exact_name}"  in row : {idx}, match score is : {score} ')

Get Game Features by given game name

In [ ]:
#selected_game_id = 1
selected_game_id = games.iloc[idx ,0]

selected_game_index = games.BGGId[games.BGGId == selected_game_id].index
#create index value for idx
#selected_game_index = pd.Int64Index(data = [idx] , dtype = 'int64')

In [ ]:
select_game_features = features.iloc[selected_game_index , 1:]

Calculate Distances

In [ ]:
distances, indices  = model_knn.kneighbors(select_game_features)

In [ ]:
#distances is 2D , convert to 1D and take all elemnts except first value(self = 1)
coff = 1- distances.round(5)[0][1:]

In [ ]:
#limit top 10 indices without the choosen game
indices = indices[0][1:11]

In [ ]:
#get relevant data of k neighbors
filter = ~games.BGGId.isin( [selected_game_id]) #without the game selected
suggested_games = games.iloc[indices ,0:6][filter == True]
suggested_games['User_relvant_score'] =  suggested_games.AvgRating * coff
suggested_games['distance'] = 1- coff
suggested_games

##Get Recommendation by User's Top 3 Rated Games

In [ ]:
#user = input('Enter your User Name :')
user = 'Tonydorrf'
#get x highest game ratings of choosen user
x = 3
highest_rank = user_rank[user_rank.Username == user].sort_values(by = 'Rating' , ascending = False).iloc[:x , 0]
#get highest_rank features
highest_rank_features = features[features.BGGId.isin(highest_rank)].iloc[:,1:]


In [ ]:
games_played = user_rank.BGGId[user_rank.Username == user]
filter = ~games.BGGId.isin( games_played) #without the games the user played

check common_features

In [ ]:
#check how many common  features all x games have
#get all columns where the value is greater than 0
col_filter = highest_rank_features.sum(axis = 0) > 0
common_features = highest_rank_features.loc[:,col_filter].columns #only 3 features exsits in at least 2 games
common_features.shape

In [ ]:
#option 2 top x features
#highest_rank_features.sum(axis = 0).sort_values(ascending = False)[:30]

In [ ]:
#Avg on highest_rank features
highest_rank_features = highest_rank_features.mean(axis = 0).rename('mean').to_frame().T
#col_filter -limit the number of features to the common ones
highest_rank_features = highest_rank_features.loc[:,common_features]

In [ ]:
features[filter == True].loc[:, common_features ]

Calculate NN

In [ ]:
#convert to sparse matrix element
features_sparse = csr_matrix( features[filter == True].loc[:, common_features ] ) #filter out games played + inculde only common_features col

In [ ]:
#define knn model parameters
model_knn_ug = NearestNeighbors(metric= 'cosine' , algorithm= 'brute' , n_neighbors= 11 )
#fit the model
model_knn_ug.fit(features_sparse)

In [ ]:
distances, indices  = model_knn_ug.kneighbors(highest_rank_features)

In [ ]:
distances

In [ ]:
#distances is 2D , convert to 1D and take first 10 elements
coff = 1- distances.round(5)[0][0:10]
coff

In [ ]:
#limit top 10 indices without the choosen game
indices = indices[0][0:10]
indices

In [ ]:
#get relevant data of k neighbors
suggested_games = games.iloc[indices ,0:6]#[filter == True]
suggested_games['User_relvant_score'] =  suggested_games.AvgRating * coff
suggested_games['distance'] = 1- coff
suggested_games

###Check 'suggested games' Relvance

In [ ]:
#filter_ = features[common_features].sum(axis = 1) == 3

#filter rows where avg usage in 'common_features' is greater than 30%
filter_ = features[common_features].mean(axis = 1) > 0.1
lis_check = features.loc[:,features.columns.isin(common_features)][filter_]

In [ ]:
#get suggested_games with common_features columns
games_check = features.loc[suggested_games.index , common_features ] #features.columns.isin(common_features)
#games_check.join(games , how= 'inner')
#filter
#games_check.loc[:,features.columns.isin(common_features)]#[filter_]

lis_check.join( games_check,how = 'inner' , lsuffix = '_left' , rsuffix = '_r')

In [ ]:
#check common features with games played
#games_palyed & filter_ = games with common features
games[ games.BGGId.isin(games_played) & filter_ ==True ].shape

### **Surprise Package - Collaborative Filtering**

![image.png](attachment:46b08395-ac9b-4cb2-913a-dfa301580a80.png)

In [ ]:
#filter users that have rated between 10 to 200 games
user_rank_filtered = user_rank.groupby('Username').filter(lambda grp: len(grp) <= 200 and len(grp) >= 10 )
print(user_rank_filtered.shape)
user_rank_filtered.head(1)

##Define Functions for Results

In [ ]:
def get_Iu(uid):
    """ return the number of items rated by given user
    args:
      uid: the id of the user
    returns:
      the number of items rated by the user
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0

def get_Ui(iid):
    """ return number of users that have rated given item
    args:
      iid: the raw id of the item
    returns:
      the number of users that have rated the item.
    """
    try:
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0


In [ ]:
from collections import defaultdict

def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

## Specif User Sample - BaselineOnly Algorithm

In [ ]:
#choose user to suggest to
active_user_id = 'zzzvone'
active_user_ratings = user_rank[user_rank['Username'] == active_user_id]

#get active_user_games and users that played same games as the active user
active_user_games = active_user_ratings['BGGId']
common_ratings = user_rank[user_rank['BGGId'].isin(active_user_games)]

#filter users that have rated at least 'min_common_items'(15) from 'active user' games
min_common_items = 15
reduce_rating = common_ratings.groupby('Username').filter(lambda grp: len(grp) >= min_common_items)

In [ ]:
#surprise reader object
reader = Reader(rating_scale=(0, 10))
#data = Dataset.load_from_df(df_new[['userID', 'ISBN', 'bookRating']], reader)
data = Dataset.load_from_df(reduce_rating[['Username', 'BGGId' , 'Rating']], reader)
data

In [ ]:
#define algorithm params and get cross_validate scores
print('Using ALS')
bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
algo = BaselineOnly(bsl_options=bsl_options)
cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)

In [ ]:
#rain ,test split
trainset, testset = sup_tt_split( data, test_size=0.25)
#define algorithm
algo = BaselineOnly(bsl_options=bsl_options)
#fit - predict
predictions = algo.fit(trainset).test(testset)
#RMSE of predictions
accuracy.rmse(predictions)

In [ ]:
trainset = algo.trainset
print(algo.__class__.__name__)

In [ ]:
#add columns to predictions df - # of items ratigs , # of user ratings ,error diff
df = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
df['Iu_item_freq'] = df.uid.apply(get_Iu)
df['Ui_user_freq'] = df.iid.apply(get_Ui)
df['err'] = abs(df.est - df.rui)

In [ ]:
df.head()

In [ ]:
# bringin the worst and best predictions
best_predictions = df.sort_values(by='err')[:10]
worst_predictions = df.sort_values(by='err')[-10:]

In [ ]:
best_predictions

In [ ]:
worst_predictions

In [ ]:
#distribution of error
df.err.sort_values().hist( bins = 100, density = True)

In [ ]:
# using top N fanction to pick for  user
top_n = get_top_n(predictions, n= 10)

In [ ]:
# gatting the game in to a list
result = top_n[active_user_id]
game_id , rating = zip(*result)

In [ ]:
#bringing top games info
top_games = games[games['BGGId'].isin(game_id)]
# format top games info
top_games = top_games[['BGGId','Name','Description','ImagePath']]

##Random Sample Find Best Algorithm

In [ ]:
#take a sample of 1000 users
sample_users = pd.Series(user_rank_filtered['Username'].unique()).sample(2000) #user_rank_filtered['Username'].unique()[:2000]
sample_users = user_rank_filtered[user_rank_filtered.Username.isin(sample_users)]
sample_users.shape

In [ ]:
#surprise reader object
reader = Reader(rating_scale=(0, 10))
#data = Dataset.load_from_df(df_new[['userID', 'ISBN', 'bookRating']], reader)
data = Dataset.load_from_df(sample_users[['Username', 'BGGId' , 'Rating']], reader)
data

In [ ]:
benchmark = []
# Iterate over all algorithms
for algorithm in [ SVD(), SVDpp(), KNNBaseline(), KNNBasic(), BaselineOnly()]: #NMF() ,SlopeOne(),  CoClustering() ,NormalPredictor(), KNNWithMeans(), KNNWithZScore(), #
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False ,return_train_measures = True)
    print(results)

    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)

In [ ]:
#surprise_results
surprise_results = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')
surprise_results['dif_train_test'] = surprise_results.test_rmse - surprise_results.train_rmse
surprise_results

### Matrix factorization explain

![image.png](attachment:31c2a367-6149-46be-81dd-2d70f4c7f72a.png)

In [ ]:
#SVDpp is the best , but the dif_train_test is 5 time bigger
surprise_results.loc[surprise_results.test_rmse.idxmin()]

In [ ]:
#define algorithm params and get cross_validate scores
print('Using ALS')
bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
algo = BaselineOnly(bsl_options=bsl_options)
cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)

In [ ]:
#rain ,test split
trainset, testset = sup_tt_split( data, test_size=0.25)
#define algorithm
algo = BaselineOnly(bsl_options=bsl_options)
#fit - predict
predictions = algo.fit(trainset).test(testset)
#RMSE of predictions
accuracy.rmse(predictions)

In [ ]:
trainset = algo.trainset
print(algo.__class__.__name__)

In [ ]:
#add columns to predictions df - # of items ratigs , # of user ratings
df = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
df['Iu_item_freq'] = df.uid.apply(get_Iu)
df['Ui_user_freq'] = df.iid.apply(get_Ui)
df['err'] = abs(df.est - df.rui)

In [ ]:
# bringin the worst and best predictions
best_predictions = df.sort_values(by='err')[:10]
worst_predictions = df.sort_values(by='err')[-10:]

In [ ]:
best_predictions

In [ ]:
worst_predictions

In [ ]:
#distribution of error

df.err.sort_values().hist( bins = 100, density = True)

In [ ]:
#cumulative % of error
from matplotlib.ticker import PercentFormatter
df.err.sort_values().plot( kind = 'hist' , bins = 100
                          ,fontsize = 13
                          ,xticks = range(0,10) ,histtype='step',
                           cumulative=True ,weights=np.ones(len(df)) / len(df) , figsize = (10,8) )

plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.gca().set_xlabel("RMSE", fontsize = 20)
plt.gca().set_ylabel( 'cumulative %', fontsize = 20)
plt.gca().set_title('Cumulative % of RMSE' , fontsize = 20)
